# 1. 데이터 불러오기 

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

In [3]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train.head()

,ID,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,...,기증 배아 사용 여부,대리모 여부,PGD 시술 여부,PGS 시술 여부,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일,임신 성공 여부
0,TRAIN_000000,TRZKPL,만18-34세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,0
1,TRAIN_000001,TRYBLT,만45-50세,NaN,IVF,ICSI,0,알 수 없음,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0
2,TRAIN_000002,TRVNRY,만18-34세,NaN,IVF,IVF,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,2.0,NaN,0
3,TRAIN_000003,TRJXFG,만35-37세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0
4,TRAIN_000004,TRVNRY,만18-34세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,0


# 2. 데이터 결측치 확인 & 제거 

In [4]:
# 전체 값 개수 & 결측치 개수 출력
missing_info = pd.DataFrame({
    "전체 값 개수": train.shape[0],  # 전체 행 개수
    "Non-값 개수": train.isnull().sum(),  # 결측치 개수
    "결측치 비율(%)": (train.isnull().sum() / train.shape[0]) * 100  # 결측치 비율(%)
})

# 변환 결과 확인
pd.set_option("display.max_rows", None)  # 모든 행 출력
pd.set_option("display.max_columns", None)  # 모든 열 출력
pd.set_option("display.max_colwidth", None)  # 열의 최대 너비 제한 해제
pd.set_option("display.expand_frame_repr", False)  # 너비에 따라 자동 줄바꿈 방지

#print(missing_info)

# 결측치가 있는 열만 출력
missing_info[missing_info["결측치 비율(%)"] > 20]

,전체 값 개수,Non-값 개수,결측치 비율(%)
임신 시도 또는 마지막 임신 경과 연수,256351,246981,96.344855
착상 전 유전 검사 사용 여부,256351,253633,98.939735
PGD 시술 여부,256351,254172,99.149994
PGS 시술 여부,256351,254422,99.247516
난자 채취 경과일,256351,57488,22.425503
난자 해동 경과일,256351,254915,99.439831
난자 혼합 경과일,256351,53735,20.961494
배아 해동 경과일,256351,215982,84.252451


In [5]:
# 결측치 비율이 80% 이상인 열을 찾기
cols_to_drop = missing_info[missing_info["결측치 비율(%)"] >= 80].index

# 해당 열 제거
train = train.drop(columns=cols_to_drop)

# 변경된 데이터프레임 확인
print(f"제거된 열: {list(cols_to_drop)}") # 5

print(train.columns)
print(len(train.columns))

제거된 열: ['임신 시도 또는 마지막 임신 경과 연수', '착상 전 유전 검사 사용 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
Index(['ID', '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 자극 여부',
       '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인',
       '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
       '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애',
       '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도',
       '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유',
       '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수',
       'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수',
       '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수',
       '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',
       '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
       '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이',
       '동결 배아 사용 여부', '신선 배아 사용 

In [6]:
train = train.drop(columns='시술 유형')


# 3. 범주형, 텍스트형 데이터를 라벨링

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
# 범주형 및 텍스트형 열 찾기
categorical_cols = train.select_dtypes(include=["object"]).columns  # object 타입(범주형, 문자열) 찾기

# Label Encoding 적용
label_encoders = {}  # 각 열의 인코더를 저장할 딕셔너리
for col in categorical_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))  # 문자열로 변환 후 인코딩
    label_encoders[col] = le  # 인코더 저장 (나중에 디코딩할 수도 있음)

# 변환된 데이터 확인
print(train.head())


   ID  시술 시기 코드  시술 당시 나이  특정 시술 유형  배란 자극 여부  배란 유도 유형  단일 배아 이식 여부  착상 전 유전 진단 사용 여부  남성 주 불임 원인  남성 부 불임 원인  여성 주 불임 원인  여성 부 불임 원인  부부 주 불임 원인  부부 부 불임 원인  불명확 불임 원인  불임 원인 - 난관 질환  불임 원인 - 남성 요인  불임 원인 - 배란 장애  불임 원인 - 여성 요인  불임 원인 - 자궁경부 문제  불임 원인 - 자궁내막증  불임 원인 - 정자 농도  불임 원인 - 정자 면역학적 요인  불임 원인 - 정자 운동성  불임 원인 - 정자 형태  배아 생성 주요 이유  총 시술 횟수  클리닉 내 총 시술 횟수  IVF 시술 횟수  DI 시술 횟수  총 임신 횟수  IVF 임신 횟수  DI 임신 횟수  총 출산 횟수  IVF 출산 횟수  DI 출산 횟수  총 생성 배아 수  미세주입된 난자 수  미세주입에서 생성된 배아 수  이식된 배아 수  미세주입 배아 이식 수  저장된 배아 수  미세주입 후 저장된 배아 수  해동된 배아 수  해동 난자 수  수집된 신선 난자 수  저장된 신선 난자 수  혼합된 난자 수  파트너 정자와 혼합된 난자 수  기증자 정자와 혼합된 난자 수  난자 출처  정자 출처  난자 기증자 나이  정자 기증자 나이  동결 배아 사용 여부  신선 배아 사용 여부  기증 배아 사용 여부  대리모 여부  난자 채취 경과일  난자 혼합 경과일  배아 이식 경과일  임신 성공 여부
0   0         6         0         4         1         0          0.0               0.0           0           0           0           0           0           0          0              0              1              1              0                0

# 4. 결측값이 있는 행 처리 

In [9]:
# 총 행 수
total_rows = train.shape[0]

# 결측치가 하나라도 있는 행의 개수
rows_with_missing_values = train.isnull().any(axis=1).sum()

print(f"총 행 수: {total_rows}")
print(f"결측치가 하나라도 있는 행의 개수: {rows_with_missing_values}")

총 행 수: 256351
결측치가 하나라도 있는 행의 개수: 90554


In [10]:
# 각 열별 결측치 개수 세기
missing_values = train.isnull().sum()

print("각 열별 결측치 개수:")
print(missing_values)

각 열별 결측치 개수:
ID                        0
시술 시기 코드                  0
시술 당시 나이                  0
특정 시술 유형                  0
배란 자극 여부                  0
배란 유도 유형                  0
단일 배아 이식 여부            6291
착상 전 유전 진단 사용 여부       6291
남성 주 불임 원인                0
남성 부 불임 원인                0
여성 주 불임 원인                0
여성 부 불임 원인                0
부부 주 불임 원인                0
부부 부 불임 원인                0
불명확 불임 원인                 0
불임 원인 - 난관 질환             0
불임 원인 - 남성 요인             0
불임 원인 - 배란 장애             0
불임 원인 - 여성 요인             0
불임 원인 - 자궁경부 문제           0
불임 원인 - 자궁내막증             0
불임 원인 - 정자 농도             0
불임 원인 - 정자 면역학적 요인        0
불임 원인 - 정자 운동성            0
불임 원인 - 정자 형태             0
배아 생성 주요 이유               0
총 시술 횟수                   0
클리닉 내 총 시술 횟수             0
IVF 시술 횟수                 0
DI 시술 횟수                  0
총 임신 횟수                   0
IVF 임신 횟수                 0
DI 임신 횟수                  0
총 출산 횟수                   0
IVF 출산 횟수                 0
DI 출산 횟

난자 채취 경과일, 난자 혼합 경과일, 배아 이식 경과일에 대해서는 평균값으로 대체
(학습시켜서 예측값으로 대체할 수도 있음 ) 

In [11]:
# 중앙값으로 결측치를 대체
train['난자 채취 경과일'].fillna(train['난자 채취 경과일'].median(), inplace=True)
train['난자 혼합 경과일'].fillna(train['난자 혼합 경과일'].median(), inplace=True)
train['배아 이식 경과일'].fillna(train['배아 이식 경과일'].median(), inplace=True)

# 결측치가 대체되었는지 확인
print(train[['난자 채취 경과일','난자 혼합 경과일', '배아 이식 경과일']].isnull().sum())

난자 채취 경과일    0
난자 혼합 경과일    0
배아 이식 경과일    0
dtype: int64


In [12]:
# 각 열별 결측치 개수 세기
missing_values = train.isnull().sum()

print("각 열별 결측치 개수:")
print(missing_values)

각 열별 결측치 개수:
ID                       0
시술 시기 코드                 0
시술 당시 나이                 0
특정 시술 유형                 0
배란 자극 여부                 0
배란 유도 유형                 0
단일 배아 이식 여부           6291
착상 전 유전 진단 사용 여부      6291
남성 주 불임 원인               0
남성 부 불임 원인               0
여성 주 불임 원인               0
여성 부 불임 원인               0
부부 주 불임 원인               0
부부 부 불임 원인               0
불명확 불임 원인                0
불임 원인 - 난관 질환            0
불임 원인 - 남성 요인            0
불임 원인 - 배란 장애            0
불임 원인 - 여성 요인            0
불임 원인 - 자궁경부 문제          0
불임 원인 - 자궁내막증            0
불임 원인 - 정자 농도            0
불임 원인 - 정자 면역학적 요인       0
불임 원인 - 정자 운동성           0
불임 원인 - 정자 형태            0
배아 생성 주요 이유              0
총 시술 횟수                  0
클리닉 내 총 시술 횟수            0
IVF 시술 횟수                0
DI 시술 횟수                 0
총 임신 횟수                  0
IVF 임신 횟수                0
DI 임신 횟수                 0
총 출산 횟수                  0
IVF 출산 횟수                0
DI 출산 횟수                 0
총 생성 배아 수      

6291은 모두 시술유형 - DI에 대한 값임 

# 5. 상관관계 분석

# 6. 라벨 분포 시각화 

In [13]:
# 각 피쳐에서 고유값 확인
unique_values = train.apply(lambda x: x.unique())

for col, values in unique_values.items():
    print(f"{col}: {values}")

ID: [     0      1      2 ... 256348 256349 256350]
시술 시기 코드: [6 5 3 2 4 0 1]
시술 당시 나이: [0 5 1 2 3 4 6]
특정 시술 유형: [ 4 15 23 14 20 18 12  5 11 19  7 21 13 16 22  2  9  3 17 24  1 10  6  0
  8]
배란 자극 여부: [1 0]
배란 유도 유형: [0 3 2 1]
단일 배아 이식 여부: [ 0.  1. nan]
착상 전 유전 진단 사용 여부: [ 0. nan  1.]
남성 주 불임 원인: [0 1]
남성 부 불임 원인: [0 1]
여성 주 불임 원인: [0 1]
여성 부 불임 원인: [0 1]
부부 주 불임 원인: [0 1]
부부 부 불임 원인: [0 1]
불명확 불임 원인: [0 1]
불임 원인 - 난관 질환: [0 1]
불임 원인 - 남성 요인: [1 0]
불임 원인 - 배란 장애: [1 0]
불임 원인 - 여성 요인: [0]
불임 원인 - 자궁경부 문제: [0 1]
불임 원인 - 자궁내막증: [0 1]
불임 원인 - 정자 농도: [0 1]
불임 원인 - 정자 면역학적 요인: [0 1]
불임 원인 - 정자 운동성: [0 1]
불임 원인 - 정자 형태: [0 1]
배아 생성 주요 이유: [13  0  6 10  5  3  1  2 11  7  4 12  9  8]
총 시술 횟수: [0 1 2 6 3 4 5]
클리닉 내 총 시술 횟수: [0 1 2 6 3 5 4]
IVF 시술 횟수: [0 1 2 3 5 4 6]
DI 시술 횟수: [0 5 3 1 2 4 6]
총 임신 횟수: [0 1 2 3 4 5 6]
IVF 임신 횟수: [0 1 2 3 4 5 6]
DI 임신 횟수: [0 1 2 3 4 5]
총 출산 횟수: [0 1 2 3 4 6 5]
IVF 출산 횟수: [0 1 2 3 4 5]
DI 출산 횟수: [0 1 2 3 4]
총 생성 배아 수: [ 4.  0.  5.  6.  3. 12.  2. 13. nan  8.  7.  1

In [14]:
# ID 열을 제외한 각 열별로 값의 개수와 비율을 구하는 코드
result = {}

# ID 열을 제외한 열만 선택
for col in train.columns:
    if col != 'ID':  # 'ID' 열 제외
        value_counts = train[col].value_counts(dropna=False)  # 결측치도 포함해서 계산
        percent = (value_counts / len(train)) * 100  # 각 값이 차지하는 비율 계산
        
        # 결과 저장
        result[col] = pd.DataFrame({
            'count': value_counts,
            'percent': percent
        })

# 결과 출력 (각 열별로 개수와 비율 출력)
for col, data in result.items():
    print(f"Column: {col}")
    print(data)
    print("\n")


Column: 시술 시기 코드
          count    percent
시술 시기 코드                  
1         38969  15.201423
0         38090  14.858534
5         36713  14.321380
3         36173  14.110731
2         36031  14.055338
6         35544  13.865364
4         34831  13.587230


Column: 시술 당시 나이
           count    percent
시술 당시 나이                   
0         102476  39.974878
1          57780  22.539409
2          39247  15.309868
3          37348  14.569087
4          12253   4.779775
5           6918   2.698644
6            329   0.128340


Column: 특정 시술 유형
           count    percent
특정 시술 유형                   
4         122368  47.734551
15         91755  35.792722
23         26939  10.508639
14          6100   2.379550
11          2314   0.902669
7           1609   0.627655
18          1248   0.486833
20          1146   0.447043
12           873   0.340549
5            769   0.299979
19           392   0.152915
16           319   0.124439
13           207   0.080749
21           100   0.039009
3 

In [15]:
# 각 열에 대해 범주별 비율 확인
def check_category_imbalance(df):
    imbalance_info = {}
    for column in train.columns:
        if column != 'ID':  # ID 열 제외
            value_counts = train[column].value_counts(normalize=True) * 100  # 비율
            imbalance_info[column] = value_counts
    return imbalance_info

# 불균형 데이터 확인
imbalance_stats = check_category_imbalance(train)
for col, counts in imbalance_stats.items():
    print(f"{col}의 범주 비율:")
    print(counts)
    print("\n")

시술 시기 코드의 범주 비율:
시술 시기 코드
1    15.201423
0    14.858534
5    14.321380
3    14.110731
2    14.055338
6    13.865364
4    13.587230
Name: proportion, dtype: float64


시술 당시 나이의 범주 비율:
시술 당시 나이
0    39.974878
1    22.539409
2    15.309868
3    14.569087
4     4.779775
5     2.698644
6     0.128340
Name: proportion, dtype: float64


특정 시술 유형의 범주 비율:
특정 시술 유형
4     47.734551
15    35.792722
23    10.508639
14     2.379550
11     0.902669
7      0.627655
18     0.486833
20     0.447043
12     0.340549
5      0.299979
19     0.152915
16     0.124439
13     0.080749
21     0.039009
3      0.037449
2      0.027696
22     0.008972
10     0.002341
9      0.001560
0      0.001170
17     0.000780
24     0.000780
1      0.000780
6      0.000780
8      0.000390
Name: proportion, dtype: float64


배란 자극 여부의 범주 비율:
배란 자극 여부
1    77.128624
0    22.871376
Name: proportion, dtype: float64


배란 유도 유형의 범주 비율:
배란 유도 유형
0    75.846008
3    24.153212
2     0.000390
1     0.000390
Name: proportion, dtype: float

In [16]:
# 데이터 비율이 99% 이상인 열 제거 
train = train.drop(columns='부부 부 불임 원인')
train = train.drop(columns='불임 원인 - 여성 요인')
train = train.drop(columns='불임 원인 - 자궁경부 문제')
train = train.drop(columns='불임 원인 - 정자 농도')
train = train.drop(columns='불임 원인 - 정자 면역학적 요인')
train = train.drop(columns='불임 원인 - 정자 운동성')
train = train.drop(columns='불임 원인 - 정자 형태')
train = train.drop(columns='DI 출산 횟수')
train = train.drop(columns='난자 채취 경과일')
train = train.drop(columns='난자 혼합 경과일')
train = train.drop(columns='해동 난자 수')
train = train.drop(columns='저장된 신선 난자 수')
train = train.drop(columns='기증자 정자와 혼합된 난자 수')
train = train.drop(columns='남성 부 불임 원인')
train = train.drop(columns='여성 주 불임 원인')
train = train.drop(columns='여성 부 불임 원인')
train = train.drop(columns='부부 주 불임 원인')

In [17]:
# 각 열에 대해 범주별 비율 확인
def check_category_imbalance(df):
    imbalance_info = {}
    for column in train.columns:
        if column != 'ID':  # ID 열 제외
            value_counts = train[column].value_counts(normalize=True) * 100  # 비율
            imbalance_info[column] = value_counts
    return imbalance_info

# 불균형 데이터 확인
imbalance_stats = check_category_imbalance(train)
for col, counts in imbalance_stats.items():
    print(f"{col}의 범주 비율:")
    print(counts)
    print("\n")

시술 시기 코드의 범주 비율:
시술 시기 코드
1    15.201423
0    14.858534
5    14.321380
3    14.110731
2    14.055338
6    13.865364
4    13.587230
Name: proportion, dtype: float64


시술 당시 나이의 범주 비율:
시술 당시 나이
0    39.974878
1    22.539409
2    15.309868
3    14.569087
4     4.779775
5     2.698644
6     0.128340
Name: proportion, dtype: float64


특정 시술 유형의 범주 비율:
특정 시술 유형
4     47.734551
15    35.792722
23    10.508639
14     2.379550
11     0.902669
7      0.627655
18     0.486833
20     0.447043
12     0.340549
5      0.299979
19     0.152915
16     0.124439
13     0.080749
21     0.039009
3      0.037449
2      0.027696
22     0.008972
10     0.002341
9      0.001560
0      0.001170
17     0.000780
24     0.000780
1      0.000780
6      0.000780
8      0.000390
Name: proportion, dtype: float64


배란 자극 여부의 범주 비율:
배란 자극 여부
1    77.128624
0    22.871376
Name: proportion, dtype: float64


배란 유도 유형의 범주 비율:
배란 유도 유형
0    75.846008
3    24.153212
2     0.000390
1     0.000390
Name: proportion, dtype: float

In [18]:
train = train.drop(columns='대리모 여부')
train = train.drop(columns='기증 배아 사용 여부')
train = train.drop(columns='DI 임신 횟수')
train = train.drop(columns='착상 전 유전 진단 사용 여부')

In [19]:
# 각 열에 대해 범주별 비율 확인
def check_category_imbalance(df):
    imbalance_info = {}
    for column in train.columns:
        if column != 'ID':  # ID 열 제외
            value_counts = train[column].value_counts(normalize=True) * 100  # 비율
            imbalance_info[column] = value_counts
    return imbalance_info

# 불균형 데이터 확인
imbalance_stats = check_category_imbalance(train)
for col, counts in imbalance_stats.items():
    print(f"{col}의 범주 비율:")
    print(counts)
    print("\n")

시술 시기 코드의 범주 비율:
시술 시기 코드
1    15.201423
0    14.858534
5    14.321380
3    14.110731
2    14.055338
6    13.865364
4    13.587230
Name: proportion, dtype: float64


시술 당시 나이의 범주 비율:
시술 당시 나이
0    39.974878
1    22.539409
2    15.309868
3    14.569087
4     4.779775
5     2.698644
6     0.128340
Name: proportion, dtype: float64


특정 시술 유형의 범주 비율:
특정 시술 유형
4     47.734551
15    35.792722
23    10.508639
14     2.379550
11     0.902669
7      0.627655
18     0.486833
20     0.447043
12     0.340549
5      0.299979
19     0.152915
16     0.124439
13     0.080749
21     0.039009
3      0.037449
2      0.027696
22     0.008972
10     0.002341
9      0.001560
0      0.001170
17     0.000780
24     0.000780
1      0.000780
6      0.000780
8      0.000390
Name: proportion, dtype: float64


배란 자극 여부의 범주 비율:
배란 자극 여부
1    77.128624
0    22.871376
Name: proportion, dtype: float64


배란 유도 유형의 범주 비율:
배란 유도 유형
0    75.846008
3    24.153212
2     0.000390
1     0.000390
Name: proportion, dtype: float

In [20]:
train_filtered = train.copy()  # 원본 데이터 보호

# 각 열에 대해 상한값 적용
train_filtered['시술 당시 나이'] = train_filtered['시술 당시 나이'].clip(upper=5)
train_filtered['총 생성 배아 수'] = train_filtered['총 생성 배아 수'].clip(upper=15)
train_filtered['미세주입된 난자 수'] = train_filtered['미세주입된 난자 수'].clip(upper=15)
train_filtered['미세주입에서 생성된 배아 수'] = train_filtered['미세주입에서 생성된 배아 수'].clip(upper=11)
train_filtered['저장된 배아 수'] = train_filtered['저장된 배아 수'].clip(upper=6)
train_filtered['미세주입 후 저장된 배아 수'] = train_filtered['미세주입 후 저장된 배아 수'].clip(upper=6)
train_filtered['해동된 배아 수'] = train_filtered['해동된 배아 수'].clip(upper=5)
train_filtered['수집된 신선 난자 수'] = train_filtered['수집된 신선 난자 수'].clip(upper=21)
train_filtered['혼합된 난자 수'] = train_filtered['혼합된 난자 수'].clip(upper=18)
train_filtered['파트너 정자와 혼합된 난자 수'] = train_filtered['파트너 정자와 혼합된 난자 수'].clip(upper=19)
train_filtered['배아 이식 경과일'] = train_filtered['배아 이식 경과일'].clip(upper=4)
train_filtered['DI 시술 횟수'] = train_filtered['DI 시술 횟수'].clip(upper=5)
train_filtered['총 임신 횟수'] = train_filtered['총 임신 횟수'].clip(upper=2)
train_filtered['총 출산 횟수'] = train_filtered['총 출산 횟수'].clip(upper=1)
train_filtered['IVF 출산 횟수'] = train_filtered['IVF 출산 횟수'].clip(upper=1)



In [21]:
# 각 열에 대해 범주별 비율 확인
def check_category_imbalance(df):
    imbalance_info = {}
    for column in df.columns:
        if column != 'ID':  # ID 열 제외
            value_counts = df[column].value_counts(normalize=True) * 100  # 비율
            imbalance_info[column] = value_counts
    return imbalance_info

# 불균형 데이터 확인
imbalance_stats = check_category_imbalance(train_filtered)  # resampled_df_filtered 사용
for col, counts in imbalance_stats.items():
    print(f"{col}의 범주 비율:")
    print(counts)
    print("\n")

시술 시기 코드의 범주 비율:
시술 시기 코드
1    15.201423
0    14.858534
5    14.321380
3    14.110731
2    14.055338
6    13.865364
4    13.587230
Name: proportion, dtype: float64


시술 당시 나이의 범주 비율:
시술 당시 나이
0    39.974878
1    22.539409
2    15.309868
3    14.569087
4     4.779775
5     2.826983
Name: proportion, dtype: float64


특정 시술 유형의 범주 비율:
특정 시술 유형
4     47.734551
15    35.792722
23    10.508639
14     2.379550
11     0.902669
7      0.627655
18     0.486833
20     0.447043
12     0.340549
5      0.299979
19     0.152915
16     0.124439
13     0.080749
21     0.039009
3      0.037449
2      0.027696
22     0.008972
10     0.002341
9      0.001560
0      0.001170
17     0.000780
24     0.000780
1      0.000780
6      0.000780
8      0.000390
Name: proportion, dtype: float64


배란 자극 여부의 범주 비율:
배란 자극 여부
1    77.128624
0    22.871376
Name: proportion, dtype: float64


배란 유도 유형의 범주 비율:
배란 유도 유형
0    75.846008
3    24.153212
2     0.000390
1     0.000390
Name: proportion, dtype: float64


단일 배아 이식 여

In [22]:
print(len(train_filtered.columns))
print(train_filtered.columns)

41
Index(['ID', '시술 시기 코드', '시술 당시 나이', '특정 시술 유형', '배란 자극 여부', '배란 유도 유형',
       '단일 배아 이식 여부', '남성 주 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁내막증', '배아 생성 주요 이유',
       '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수',
       'IVF 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수',
       '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
       '미세주입 후 저장된 배아 수', '해동된 배아 수', '수집된 신선 난자 수', '혼합된 난자 수',
       '파트너 정자와 혼합된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이',
       '동결 배아 사용 여부', '신선 배아 사용 여부', '배아 이식 경과일', '임신 성공 여부'],
      dtype='object')


# 7. train 

In [23]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report

In [24]:
X = train_filtered.drop(columns=['임신 성공 여부'])  
y = train_filtered['임신 성공 여부']

# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [25]:
# XGBoost 모델 설정
model = xgb.XGBClassifier(
    n_estimators=1000,  # 트리 개수
    learning_rate=0.05,  # 학습률
    max_depth=6,  # 최대 트리 깊이
    scale_pos_weight=3,  # 클래스 불균형 조정 (성공률이 낮을 경우 조정)
    subsample=0.8,  # 샘플링 비율
    colsample_bytree=0.8,  # 컬럼 샘플링 비율
    random_state=42
)

# 모델 학습
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.57      0.69     38025
           1       0.38      0.76      0.51     13246

    accuracy                           0.62     51271
   macro avg       0.63      0.67      0.60     51271
weighted avg       0.75      0.62      0.65     51271



In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# 하이퍼파라미터 후보 설정
param_grid = {
    'n_estimators': [500, 1000, 2000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'scale_pos_weight': [1, 3, 5]  # 클래스 불균형 조정
}

# 랜덤 서치
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=20,  # 실험 횟수
    scoring='f1',  # 불균형 데이터에서 F1-score 최적화
    cv=5,  # 5-Fold 교차검증
    random_state=42,
    verbose=2,
    n_jobs=-1
)

# 최적 하이퍼파라미터 찾기
random_search.fit(X_train, y_train)

# 최적 모델 적용
best_model = random_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# 최종 평가
print(classification_report(y_test, y_pred_best))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
